In [177]:
import numpy as np
import time
np.random.seed(1337)

In [178]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Flatten,Dropout,Activation
from keras.layers import Conv1D,Convolution2D, Embedding
from keras.models import Sequential , Model
from keras.layers import Input , Concatenate ,add ,Flatten,InputLayer
from keras.optimizers import SGD
from importlib import reload
from matplotlib import pyplot
import pandas as pd
from keras.utils.vis_utils import plot_model

In [179]:
print ("start time: "+time.strftime("%Y/%m/%d  %H:%M:%S"))
distances = []  # TrainSet
labels = []  # 0/1
MAX_SEQUENCE_LENGTH = 15 
#EMBEDDING_DIM = 200 # Dimension of word vector


start time: 2020/05/30  00:25:38


In [180]:
with open('Inputdata.txt','r') as file_to_read:
    for line in file_to_read.readlines():
        values = line.rstrip().split(" ")
        distance = values[:-1]
        distances.append(distance)
        label =values[7:]
        labels.append(label)

In [181]:
print('Found %s train_distances.' % len(distances)) 

Found 884 train_distances.


In [182]:

distances = np.asarray(distances)
labels = to_categorical(np.asarray(labels))

In [183]:
n_test = 734
trainX, testX = distances[:n_test, :], distances[n_test:, :]
trainy, testy = labels[:n_test], labels[n_test:]


In [184]:
print('Found %s train_distances.' % len(trainX)) 
print('Found %s test_distances.' % len(testX)) 
print('Shape of test_label tensor:', testy.shape)
print('Shape of train_label tensor:', trainy.shape)


Found 734 train_distances.
Found 150 test_distances.
Shape of test_label tensor: (150, 8)
Shape of train_label tensor: (734, 8)


In [185]:
x_train = []
x_train_dis = trainX 
x_train_dis = np.expand_dims(x_train_dis, axis=2)
x_train.append(np.array(x_train_dis))
x_train_lbl = np.array(trainy)
print('Training model.')
print(x_train_lbl)

Training model.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]


In [186]:
y_test = []
y_test_dis = testX 
y_test_dis = np.expand_dims(y_test_dis, axis=2)
y_test.append(np.array(y_test_dis))
y_test_lbl = np.array(testy)
print(y_test_lbl)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


<h><b>Deep Neural Network</b></h>

In [187]:
M=Sequential()
M.add(Dense(128,activation='relu',input_dim=7))
M.add(Dropout(0.5))
M.add(Dense(128,activation='relu'))
M.add(Dropout(0.5))
M.add(Dense(8,activation='softmax'))
sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
M.summary()
plot_model(M, to_file='model_MLP.png', show_shapes=True, show_layer_names=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_80 (Dense)             (None, 128)               1024      
_________________________________________________________________
dropout_73 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_81 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_74 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_82 (Dense)             (None, 8)                 1032      
Total params: 18,568
Trainable params: 18,568
Non-trainable params: 0
_________________________________________________________________


In [188]:
M.compile(loss='categorical_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])

In [192]:
M.fit(trainX, trainy,epochs=500,verbose=0)

In [193]:
_,TestAcuracy=M.evaluate(testX,testy)

150/150 [==============================] - 0s 60us/step


In [195]:
TestAcuracy

0.9333333333333333

<h><b>CNN Model</b></h>

In [170]:

model_right = Sequential()
model_right.add(Conv1D(128, 1, input_shape=(7,1), padding = "same", activation='tanh'))
model_right.add(Conv1D(128, 1, activation='tanh'))
model_right.add(Conv1D(128, 1, activation='tanh'))
model_right.add(Flatten())
model_right.summary()


new_model = Sequential()
new_model.add(InputLayer(input_shape=(896,1)))
finalOutput = new_model(model_right.output)
model = Model([model_right.input],finalOutput)
fc = Dense(128,activation='tanh')(model.output)
fc2 = Dense(8,activation='softmax')(fc)
model1 = Model(inputs=model.input, outputs=fc2)
model1.summary()
plot_model(model1, to_file='model_CNN.png', show_shapes=True, show_layer_names=True)

model1.compile(loss='categorical_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_38 (Conv1D)           (None, 7, 128)            256       
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 7, 128)            16512     
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 7, 128)            16512     
_________________________________________________________________
flatten_17 (Flatten)         (None, 896)               0         
Total params: 33,280
Trainable params: 33,280
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_38_input (InputLayer) (None, 7, 1)              0         
_________________________________________________________________
conv1d

In [202]:
print ("start training:"+time.strftime("%Y/%m/%d  %H:%M:%S"))
# fit model
history=model1.fit(x_train, x_train_lbl, validation_data=(y_test, y_test_lbl), epochs=500, verbose=0)
config = model1.get_config()


start training:2020/05/30  11:31:32


In [204]:
# evaluate the model
_, train_acc = model1.evaluate(x_train, x_train_lbl, verbose=0)
_, test_acc = model1.evaluate(y_test, y_test_lbl, verbose=0)

In [205]:
print("Training Accuracy:")
print(train_acc)
print("Testing Acuracy:")
print(test_acc)


Training Accuracy:
1.0
Testing Acuracy:
0.9866666666666667


<h><b>RNN Model</b></h>

In [175]:
from keras.layers import LSTM
RM=Sequential()
RM.add(LSTM(units=50,return_sequences= True,input_shape=(7,1)))
RM.add(Dropout(0.2))
RM.add(LSTM(units=50,return_sequences= True))
RM.add(Dropout(0.2))
RM.add(LSTM(units=50,return_sequences= True))
RM.add(Dropout(0.2))
RM.add(Flatten())
RM.summary()


NM = Sequential()
NM.add(InputLayer(input_shape=(350,1)))
FO = NM(RM.output)
M1 = Model([RM.input],FO)
F = Dense(128,activation='tanh')(M1.output)
F2 = Dense(8,activation='softmax')(F)
M2 = Model(inputs=M1.input, outputs=F2)
M2.summary()
plot_model(M2, to_file='model_RNN.png', show_shapes=True, show_layer_names=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_53 (LSTM)               (None, 7, 50)             10400     
_________________________________________________________________
dropout_68 (Dropout)         (None, 7, 50)             0         
_________________________________________________________________
lstm_54 (LSTM)               (None, 7, 50)             20200     
_________________________________________________________________
dropout_69 (Dropout)         (None, 7, 50)             0         
_________________________________________________________________
lstm_55 (LSTM)               (None, 7, 50)             20200     
_________________________________________________________________
dropout_70 (Dropout)         (None, 7, 50)             0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 350)               0         
Total para

In [161]:
M2.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [162]:
M2.fit(x_train, x_train_lbl, validation_data=(y_test, y_test_lbl), epochs=2000, verbose=0)

In [164]:
_,TestAcuracy1=M2.evaluate(y_test, y_test_lbl, verbose=0)

In [165]:
TestAcuracy1

0.96

In [18]:
json_string = model1.to_json()  
open('my_model.json','w').write(json_string)  
model1.save_weights('my_model_weights1.h5') 

In [19]:
print ("end time:"+time.strftime("%Y/%m/%d  %H:%M:%S"))

end time:2020/05/22  21:57:12
